# DeepDream

DeepDream — это метод художественной обработки изображений, основанный на использовании представлений, полученных сверточными нейронными сетями.

Алгоритм DeepDream почти идентичен методу визуализации фильтров сверточных сетей, и состоит из сверточной сети, действующей в обратном направлении: выполняет градиентное восхождение по входным данным, максимизируя активацию определенного фильтра на более высоком слое сверточной сети.

1) алгоритм DeepDream пытается максимизировать активацию всех слоев, а не только определенного фильтра, тем самым смешивая визуализации большего количества признаков;

2) вы начинаете не на пустом месте, со случайных входных данных, а с имеющегося изображения, в результате получающиеся эффекты замыкаются на существующие визуальные шаблоны, искажая элементы изображения на художественный манер;

3) входные изображения обрабатываются в разных масштабах (называемых октавами), что улучшает качество визуализации.

Начнем со сверточной сети, предварительно обученной на наборе ImageNet. В Keras имеется несколько таких сетей: VGG16, VGG19, Xception, ResNet50 и т. д. Реализовать DeepDream можно с любой из них, но, как вы понимаете, выбор повлияет на характер визуализаций, потому что разные сверточные архитектуры выделяют из исходных данных разные признаки. В оригинальной версии DeepDream использовалась модель Inception, и на практике эта модель известна красиво выглядящими картинками DeepDreams, поэтому мы используем модель Inception V3, входящую в состав Keras.

In [1]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K
K.set_learning_phase(0) # Мы не будем обучать модель, поэтому выполним данную команду, чтобы запретить все операции, имеющие отношение к обучению
model = inception_v3.InceptionV3(weights='imagenet', include_top=False) #Конструирование сети Inception V3 без сверточной основы. Модель будет загру-жаться с весами, полученными в результате предварительного обучения на наборе ImageNet

87916544/87910968 [==============================] - 668s 8us/step


Теперь вычислим потери: величину, которую мы будем максимизировать в процессе градиентного восхождения. Здесь мы будем максимизировать одновременно активации всех фильтров в нескольких слоях. В данном случае максимизироваться будет взвешенная сумма L2-норм активаций набора верхних слоев. Точный набор выбранных слоев (а также их вклад в окончательное значение потерь) оказывает большое влияние на производимые визуальные эффекты, поэтому мы должны сделать эти параметры легко настраиваемыми. Нижние слои порождают геометрические шаблоны, а верхние создают эффекты, в которых можно распознать некоторые классы из набора ImageNet (например, птицы или собаки). Начнем с произвольно выбранной конфигурации, состоящей из четырех слоев

In [2]:
layer_contributions = {'mixed2': 0.2, 'mixed3': 3., 'mixed4': 2., 'mixed5': 1.5,}

# Словарь отображает имена слоев в коэффициенты, определяющие вклады слоев в потери, которые мы будем максимизировать. 
# Обратите внимание: имена слоев жестко «зашиты» во встроенное приложение Inception V3. 
# Получить список имен всех слоев можно с помощью model.summary()

In [4]:
layer_dict = dict([(layer.name, layer) for layer in model.layers]) # словарь отображающий имена слоев  в экземпляры
loss = K.variable(0.) #величина потерь определяется добавлением вклада слоя в эту скалярную переменную
for layer_name in layer_contributions: 
    coeff = layer_contributions[layer_name] 
    activation = layer_dict[layer_name].output # Получение результата слоя 
    scaling = K.prod(K.cast(K.shape(activation), 'float32')) 
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling # Добавление L2 нормы признаков слоя к потерям. Чтобы избежать влияния рамок, в подсчете потерь участвуют только пикселы, не попадающие на рамку

In [ ]:
dream = model.input #Этот тензор хранит сгенерированное изображение

grads = K.gradients(loss, dream)[0] # Вычисление градиентов изображения с учетом потерь
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) #Нормализация градиентов
outputs = [loss, grads] 
fetch_loss_and_grads = K.function([dream], outputs) #Настройка функции Keras для извлечения значения потерь и градиентов для заданного исходного изображения

def eval_loss_and_grads(x): 
    outs = fetch_loss_and_grads([x]) 
    loss_value = outs[0] 
    grad_values = outs[1] 
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None): #функция выполняет заданное число иттераций градиентного восхождения
    for i in range(iterations): 
        loss_value, grad_values = eval_loss_and_grads(x) 
        if max_loss is not None and loss_value > max_loss: 
            break 
        print('...Loss value at', i, ':', loss_value) 
        x += step * grad_values 
    return x

Выводы:
    
1) Алгоритм DeepDream состоит из сверточной сети, действующей в обратном направлении и генерирующей входные данные на основе представлений, полученных в результате обучения.

2) Получаемые результаты выглядят забавно и напоминают визуальные галлюцинации, возникающие у людей, страдающих нарушением работы зрительного отдела коры головного мозга.

3) Обратите внимание на то, что этот процесс не является специфическим для моделирования изображений или даже для сверточных сетей. Его можно применить к речи, музыке и т. д.